# COCOデータセットダウンロード

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip -n train2017.zip
!rm -f train2017.zip

!wget http://images.cocodataset.org/zips/val2017.zip
!unzip -n val2017.zip
!rm -f val2017.zip

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -n annotations_trainval2017.zip
!rm -f annotations_trainval2017.zip

# COCO minitrainダウンロード

In [ ]:
!pip install -U gdown

In [ ]:
import gdown

gdown.download('https://drive.google.com/uc?id=1lezhgY4M_Ag13w0dEzQ7x_zQ_w0ohjin', 'instances_minitrain2017.json', quiet=False)

# YOLOX環境準備

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX
%cd YOLOX
!pip install -U pip && pip install -r requirements.txt
!pip install -v -e .

# 学習データセットコピー

In [ ]:
!mkdir dataset
!mkdir dataset/images
!mkdir dataset/annotations

!cp -rf /content/train2017 dataset/images
!cp -rf /content/val2017 dataset/images
!cp -rf /content/instances_minitrain2017.json dataset/annotations
!cp -rf /content/annotations/instances_train2017.json dataset/annotations
!cp -rf /content/annotations/instances_val2017.json dataset/annotations

# YOLOX-Nano学習用ファイル準備

In [ ]:
%%writefile nano.py

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

import torch.nn as nn

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.25
        self.input_size = (416, 416)
        self.random_size = (10, 20)
        self.mosaic_scale = (0.5, 1.5)
        self.test_size = (416, 416)
        self.mosaic_prob = 0.5
        self.enable_mixup = False
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]

        self.num_classes = 80

        # Define yourself dataset path
        self.data_dir = "/content/YOLOX/dataset/images"
        # self.train_ann = "/content/YOLOX/dataset/annotations/instances_train2017.json"
        self.train_ann = "/content/YOLOX/dataset/annotations/instances_minitrain2017.json"
        self.val_ann = "/content/YOLOX/dataset/annotations/instances_val2017.json"

        self.max_epoch = 30
        self.data_num_workers = 4
        self.eval_interval = 1

    def get_model(self, sublinear=False):
        def init_yolo(M):
            for m in M.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-3
                    m.momentum = 0.03

        if "model" not in self.__dict__:
            from yolox.models import YOLOX, YOLOPAFPN, YOLOXHead
            in_channels = [256, 512, 1024]
            # NANO model use depthwise = True, which is main difference.
            backbone = YOLOPAFPN(self.depth,
                                 self.width,
                                 in_channels=in_channels,
                                 depthwise=True)
            head = YOLOXHead(self.num_classes,
                             self.width,
                             in_channels=in_channels,
                             depthwise=True)
            self.model = YOLOX(backbone, head)

        self.model.apply(init_yolo)
        self.model.head.initialize_biases(1e-2)
        return self.model

# 訓練済みモデルダウンロード

In [ ]:
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth

# 訓練

In [ ]:
!python tools/train.py \
    -f nano.py \
    -d 1 \
    -b 64 \
    --fp16 \
    -o \
    -c yolox_nano.pth

# ONNX変換

In [ ]:
!python tools/export_onnx.py \
    --output-name yolox_nano.onnx \
    -n yolox-nano \
    -f nano.py \
    -c '/content/YOLOX/YOLOX_outputs/nano/best_ckpt.pth'

In [ ]:
from google.colab import files

files.download('yolox_nano.onnx')